In [1]:
!rm -r sample_data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# %%capture
# ! pip install datasets transformers
! pip install --upgrade --force-reinstall git+https://github.com/huggingface/transformers

In [1]:
%cd MaskDetection/

/content/drive/MyDrive/MaskDetection


In [17]:
import torch
import torchvision.transforms as transforms
import os
import pandas as pd
from skimage import io
from torch.utils.data import (
    Dataset,
    DataLoader
)

from transformers import ConvNextFeatureExtractor, ConvNextForImageClassification

model_name_or_path = 'facebook/convnext-tiny-224'
feature_extractor = ConvNextFeatureExtractor.from_pretrained(model_name_or_path)

class FeatureExtractor(object):
    def __call__(self, image, target):
        sample = feature_extractor(image, return_tensors='pt')
        sample["labels"] = target
        return sample

class MaskDataset(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.annotations = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, index):
        img_path = os.path.join(self.root_dir, self.annotations.iloc[index, 0])
        image = io.imread(img_path)
        y_label = torch.tensor(int(self.annotations.iloc[index, 1]))
        
        data = self.transform(image,y_label)

        return data

train_ds = MaskDataset(
    csv_file="train.csv",
    root_dir="",
    # transform=transforms.ToTensor(),
    transform=FeatureExtractor(),
)

test_ds = MaskDataset(
    csv_file="test.csv",
    root_dir="",
    # transform=transforms.ToTensor(),
    transform=FeatureExtractor(),
)

loading feature extractor configuration file https://huggingface.co/facebook/convnext-tiny-224/resolve/main/preprocessor_config.json from cache at /root/.cache/huggingface/transformers/9680dfbfa38989323a5ae0b86951c5860c092e3aae6dc354483c075fee48c1a1.37be7274d6b5860aee104bb1fbaeb0722fec3850a85bb2557ae9491f17f89433
Feature extractor ConvNextFeatureExtractor {
  "crop_pct": 0.875,
  "do_normalize": true,
  "do_resize": true,
  "feature_extractor_type": "ConvNextFeatureExtractor",
  "image_mean": [
    0.485,
    0.456,
    0.406
  ],
  "image_std": [
    0.229,
    0.224,
    0.225
  ],
  "resample": 3,
  "size": 224
}



In [6]:
train_ds.__getitem__(0)['pixel_values'].shape

torch.Size([1, 3, 224, 224])

In [4]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")
def compute_metrics(p):
    return metric.compute(predictions=np.argmax(p.predictions, axis=1), references=p.label_ids)

Downloading:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

In [20]:
labels = ["No Mask","Mask"]

model = ConvNextForImageClassification.from_pretrained(
    model_name_or_path,
    num_labels=len(labels),
    ignore_mismatched_sizes=True,
    id2label={str(i): c for i, c in enumerate(labels)},
    label2id={c: str(i) for i, c in enumerate(labels)}
)

loading configuration file https://huggingface.co/facebook/convnext-tiny-224/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/741b590dfc3966dd77875cd496d6f9097645132a8e8106ab873b7d130da4cab9.a98bcd3c1d885a60ccf8ce2be18a604366824aabd552be8f43a9f2206d339247
Model config ConvNextConfig {
  "architectures": [
    "ConvNextForImageClassification"
  ],
  "depths": [
    3,
    3,
    9,
    3
  ],
  "drop_path_rate": 0.0,
  "hidden_act": "gelu",
  "hidden_sizes": [
    96,
    192,
    384,
    768
  ],
  "id2label": {
    "0": "No Mask",
    "1": "Mask"
  },
  "initializer_range": 0.02,
  "label2id": {
    "Mask": "1",
    "No Mask": "0"
  },
  "layer_norm_eps": 1e-12,
  "layer_scale_init_value": 1e-06,
  "model_type": "convnext",
  "num_channels": 3,
  "num_stages": 4,
  "patch_size": 4,
  "torch_dtype": "float32",
  "transformers_version": "4.17.0.dev0"
}

loading weights file https://huggingface.co/facebook/convnext-tiny-224/resolve/main/pytorch_model.bin from

In [34]:
from transformers import TrainingArguments
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
  output_dir="./convnext-tiny-mask-finetuned",
  per_device_train_batch_size=64,
  evaluation_strategy="steps",
  num_train_epochs=4,
  save_total_limit = 4, # Only last 4 models are saved. Older ones are deleted.
  fp16=True,
  save_steps=100,
  eval_steps=100,
  logging_steps=10,
  learning_rate=2e-4,
  remove_unused_columns=False,
  push_to_hub=False,
  report_to='tensorboard',
  load_best_model_at_end=True,
)

PyTorch: setting up devices


In [35]:
import torch

def collate_fn(batch):
    return {
        'pixel_values': torch.stack([x['pixel_values'][0] for x in batch]),
        'labels': torch.tensor([x['labels'] for x in batch])
    }

In [36]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=collate_fn,
    compute_metrics=compute_metrics,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    tokenizer=feature_extractor,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=3)]
)

Using amp half precision backend


In [37]:
train_results = trainer.train()
trainer.save_model()
trainer.log_metrics("train", train_results.metrics)
trainer.save_metrics("train", train_results.metrics)
trainer.save_state()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 14428
  Num Epochs = 4
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 904


Step,Training Loss,Validation Loss,Accuracy
100,0.013000,0.023499,0.994438
200,0.014000,0.015121,0.993604
300,0.002200,0.017131,0.994716
400,0.002200,0.018141,0.995829
500,0.003700,0.015090,0.996107
600,0.010200,0.016863,0.994994
700,0.000300,0.019649,0.996107
800,0.000200,0.017847,0.995551


***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./convnext-tiny-mask-finetuned/checkpoint-100
Configuration saved in ./convnext-tiny-mask-finetuned/checkpoint-100/config.json
Model weights saved in ./convnext-tiny-mask-finetuned/checkpoint-100/pytorch_model.bin
Feature extractor saved in ./convnext-tiny-mask-finetuned/checkpoint-100/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./convnext-tiny-mask-finetuned/checkpoint-200
Configuration saved in ./convnext-tiny-mask-finetuned/checkpoint-200/config.json
Model weights saved in ./convnext-tiny-mask-finetuned/checkpoint-200/pytorch_model.bin
Feature extractor saved in ./convnext-tiny-mask-finetuned/checkpoint-200/preprocessor_config.json
***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8
Saving model checkpoint to ./convnext-tiny-mask-finetuned/checkpoint-300
Configuration saved in ./convnext-tiny-

***** train metrics *****
  epoch                    =         3.54
  total_flos               = 1195651761GF
  train_loss               =       0.0079
  train_runtime            =   1:08:20.25
  train_samples_per_second =       14.075
  train_steps_per_second   =         0.22


In [38]:
metrics = trainer.evaluate(test_ds)
trainer.log_metrics("eval", metrics)
trainer.save_metrics("eval", metrics)

***** Running Evaluation *****
  Num examples = 3596
  Batch size = 8


***** eval metrics *****
  epoch                   =       3.54
  eval_accuracy           =     0.9961
  eval_loss               =     0.0151
  eval_runtime            = 0:01:23.47
  eval_samples_per_second =     43.079
  eval_steps_per_second   =      5.391
